# Completions VS Chat Completions
### Completions
-  Instead of the input being a list of messages, the input is a freeform text string called a prompt.
- Completions uses the `https://api.openai.com/v1/completions` endpoint. 
- Completions is legacy, meaning it will no longer recieve updates but you still can use it. https://platform.openai.com/docs/guides/text-generation/completions-api
- Outdated models.


### Chat Completions
- Chat models use a list of messages as input and generate a response. They are great for both multi-turn conversations and single-turn tasks.
- Chat Completions uses `https://api.openai.com/v1/chat/completions` endpoint.
- It's the standard, it's basically used in the official documentation all the time. https://platform.openai.com/docs/quickstart/step-3-sending-your-first-api-request
- Latest models and future models will be based on chat completions.

### TL;DR
- OpenAI announced that they are discontinuing the development of the Completions API. It has been replaced by the Chat Completions API, which was launched in March 2023. https://openai.com/index/gpt-4-api-general-availability/
- Completions and Chat Completion each have their own models, which are not interchangeable. https://platform.openai.com/docs/models/model-endpoint-compatibility

# Chat Completions API
- Chat models use a list of messages as input and generate a response. They are great for both multi-turn conversations and single-turn tasks.

Initilize the connection to OpenAI:

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# Load the .env
load_dotenv(find_dotenv())
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))


An example Chat Completions API call looks like the following:

In [3]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Greetings from the village!"},
  ]
)

print("Response: ", response.choices[0].message.content)


Response:  Hello! How can I assist you today?


 - The primary input is the `messages` parameter, which must be an array of message objects.
 - Each message object should include a role (either "system", "user", or "assistant") and content.
 - Conversations can range from a single message to multiple exchanges between users and assistants.
 - Typically, conversations begin with a system message, followed by alternating user and assistant messages.
 - The sampling for `temperature` parameter is typically set between 0 and 1. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. 
  - Read more about the request body and response object of the Chat Completion API : https://platform.openai.com/docs/api-reference/chat/create

### helper function
Let's create a simple custom helper function for sending requests to the Chat Completions API. It will make it easier to use prompts and look at the generated outputs.

In [34]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [36]:
print("Response: ", get_completion("Greetings from the village!"))

Response:  Hello! How can I assist you today?


## Understanding the roles: System, User, Assistant
 - The system role is used to define the behavior of the assistant, such as modifying its personality or providing specific instructions. Note that the system message is optional, and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
 - User role contain previous and current requests or comments for the assistant to respond to.
 - Assistant role not only store previous responses but can also be crafted by the user to illustrate desired behavior.

### System role vs User role

According to the official documentation, the system role is optional, meaning we can ignore it and just use the user role to provide instructions. Read more: https://platform.openai.com/docs/guides/text-generation/chat-completions-api

So, instead of using system role like this:

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "system", "content": "Your name is Mashiro, you are a helpful assistant"}, # system role
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

print("Response: ", get_completion("What is your name?"))

Response:  Hello! My name is Mashiro. How can I assist you today?


We use the user role like this:

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": "Your name is Mashiro, you are a helpful assistant"}, # changed to user role
                {"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

print("Response: ", get_completion("What is your name?"))

Response:  Hello! My name is Mashiro. How can I assist you today?


Personally, I'd say we should stick to using the system role to set the desired behavior of the LLM. Primarly to keep the prompt organized. For developers, we use the system prompt to prime the LLM model with context, and we use the user role for user inputs.

Also, using the system role is the standard and widely used in the official documentation, including in the prompt engineering best practices. https://platform.openai.com/docs/guides/prompt-engineering/six-strategies-for-getting-better-results

Try experimenting with the roles, read more:
- https://community.openai.com/t/what-exactly-does-a-system-msg-do/459409
- https://community.openai.com/t/what-is-the-difference-between-putting-the-ai-personality-in-system-content-and-in-user-content/194938
- https://community.openai.com/t/what-should-be-included-in-the-system-part-of-the-prompt/515763
- https://community.openai.com/t/the-role-of-system-prompts/149342
- https://community.openai.com/t/the-system-role-how-it-influences-the-chat-behavior/87353
- https://community.openai.com/t/playground-default-system-message/207937 